# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [14]:
import re
import spacy
from collections import defaultdict, Counter

# Función para contar la palabra "entonces"
def contar_palabra_entonces(archivo):
    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            texto = file.read()
        texto = texto.lower()
        coincidencias = re.findall(r'\bentonces\b', texto)
        return len(coincidencias)
    except FileNotFoundError:
        print("El archivo no se encontró. Asegúrate de que la ruta sea correcta.")
        return None

# Función para identificar pronombres inclusivos asociados a candidatos
def identificar_pronombres_por_candidato(archivo):
    try:
        nlp = spacy.load("es_core_news_sm")  # Cargamos el modelo en español
        with open(archivo, 'r', encoding='utf-8') as file:
            texto = file.read()

        doc = nlp(texto)

        # Diccionario con patrones para identificar candidatos
        patrones_candidatos = {
            "Sergio Fajardo": [r'\bSergio Fajardo\b', r'\[SF\]'],
            "Germán Vargas Lleras": [r'\bGermán Vargas\b', r'\[GV\]'],
            "Iván Duque": [r'\bIván Duque\b', r'\[ID\]'],
            "Gustavo Petro": [r'\bGustavo Petro\b', r'\[GP\]']
        }

        # Diccionario con pronombres correctos e incorrectos
        pronombres = {
            "él": ["él", "el"],
            "tú": ["tú", "tu"],
            "nosotros": ["nosotros", "nosotrxs", "nosotres"],
            "ustedes": ["ustedes"],
            "ellos": ["ellos", "ellxs", "elles"],
            "ellas": ["ellas", "ellxs"]
        }

        # Asociar pronombres a candidatos
        pronombres_por_candidato = defaultdict(Counter)
        candidato_actual = None

        for token in doc:
            palabra = token.text.lower()

            # Determinar el candidato actual según los patrones
            for candidato, patrones in patrones_candidatos.items():
                for patron in patrones:
                    if re.search(patron, token.sent.text, re.IGNORECASE):
                        candidato_actual = candidato
                        break

            # Asociar el pronombre al candidato actual
            if candidato_actual:
                for pronombre_correcto, variantes in pronombres.items():
                    if palabra in variantes:
                        pronombres_por_candidato[candidato_actual][pronombre_correcto] += 1

        return pronombres_por_candidato
    except FileNotFoundError:
        print("El archivo no se encontró. Asegúrate de que la ruta sea correcta.")
        return defaultdict(Counter)

# Nombre del archivo
nombre_archivo = "debate_presidencial.txt"

# Conteo de la palabra "entonces"
cantidad_entonces = contar_palabra_entonces(nombre_archivo)

# Identificación de pronombres por candidato
pronombres_candidatos = identificar_pronombres_por_candidato(nombre_archivo)

# Impresión de resultados
print("### Lista de conteo de palabras ###")
if cantidad_entonces is not None:
    print(f'Se repitió la palabra "entonces" {cantidad_entonces} veces.')

print("\n### Identificación de temas clave ###")
for candidato, conteo_pronombres in pronombres_candidatos.items():
    print(f"\n{candidato}:")
    for pronombre, cantidad in conteo_pronombres.items():
        print(f"- {pronombre}: {cantidad} veces")

### Lista de conteo de palabras ###
Se repitió la palabra "entonces" 3 veces.

### Identificación de temas clave ###

Gustavo Petro:
- él: 101 veces
- ustedes: 12 veces
- ellos: 2 veces
- nosotros: 5 veces
- tú: 1 veces

Sergio Fajardo:
- él: 72 veces
- ustedes: 9 veces
- nosotros: 16 veces

Germán Vargas Lleras:
- él: 107 veces
- nosotros: 4 veces
- ellos: 1 veces
- ellas: 1 veces
- ustedes: 1 veces

Iván Duque:
- él: 74 veces
- ustedes: 6 veces
- nosotros: 6 veces
- ellos: 1 veces
